In [1]:
import pandas as pd

In [2]:
annotations = pd.read_json("temp_head_labeling_label_studio_2025-02-11-17-05-e95fdb59.json", orient="records")
annotations.head()

,id,annotations,file_upload,drafts,predictions,data,meta,created_at,updated_at,inner_id,total_annotations,cancelled_annotations,total_predictions,comment_count,unresolved_comment_count,last_comment_updated_at,project,updated_by,comment_authors
0,4702,"[{'id': 17, 'completed_by': 1, 'result': [{'va...",17085618-temp_head_labeling_bool_as_str.csv,[],[],{'question': 'How many years did Art Carney as...,{},2025-02-11 11:45:36.869253+00:00,2025-02-11 11:46:20.843612+00:00,1,1,0,0,0,0,NaT,3,1,[]
1,4703,"[{'id': 18, 'completed_by': 1, 'result': [{'va...",17085618-temp_head_labeling_bool_as_str.csv,[],[],{'question': 'How many total years was Art Car...,{},2025-02-11 11:45:36.869297+00:00,2025-02-11 11:46:24.061761+00:00,2,1,0,0,0,0,NaT,3,1,[]
2,4704,"[{'id': 19, 'completed_by': 1, 'result': [], '...",17085618-temp_head_labeling_bool_as_str.csv,[],[],{'question': 'Which spouse was Art Carney marr...,{},2025-02-11 11:45:36.869325+00:00,2025-02-11 11:46:34.905627+00:00,3,0,1,0,0,0,NaT,3,1,[]
3,4705,"[{'id': 20, 'completed_by': 1, 'result': [{'va...",17085618-temp_head_labeling_bool_as_str.csv,[],[],{'question': 'How many years before he died wa...,{},2025-02-11 11:45:36.869349+00:00,2025-02-11 11:46:54.025841+00:00,4,1,0,0,0,0,NaT,3,1,[]
4,4706,"[{'id': 21, 'completed_by': 1, 'result': [{'va...",17085618-temp_head_labeling_bool_as_str.csv,[],[],{'question': 'How old was Art Carney when he f...,{},2025-02-11 11:45:36.869376+00:00,2025-02-11 11:46:56.820767+00:00,5,1,0,0,0,0,NaT,3,1,[]


In [3]:
annotations = annotations.loc[:, ["annotations", "data"]]

In [4]:
annotations["data"].iloc[0]

{'question': 'How many years did Art Carney as actor since 1939?',
 'answer': '54 Years',
 'table_id': 2,
 'answer_type': 'TEMPORAL',
 'is_temporal': 'Is temporal'}

In [5]:
data_columns = list(annotations["data"].iloc[0].keys())

In [6]:
{key: lambda x: x["data"].apply(lambda y: y[key]) for key in data_columns}

{'question': <function __main__.<dictcomp>.<lambda>(x)>,
 'answer': <function __main__.<dictcomp>.<lambda>(x)>,
 'table_id': <function __main__.<dictcomp>.<lambda>(x)>,
 'answer_type': <function __main__.<dictcomp>.<lambda>(x)>,
 'is_temporal': <function __main__.<dictcomp>.<lambda>(x)>}

In [7]:
# Add key to avoid gotcha from lambda that causes late-binding closure 
# where the key variable is bound at execution not at definition time. 
annotations = annotations.assign(
    **{key: lambda x, k=key: x["data"].apply(lambda y: y[k]) for key in data_columns}
).drop(columns="data")
annotations.head()

,annotations,question,answer,table_id,answer_type,is_temporal
0,"[{'id': 17, 'completed_by': 1, 'result': [{'va...",How many years did Art Carney as actor since 1...,54 Years,2,TEMPORAL,Is temporal
1,"[{'id': 18, 'completed_by': 1, 'result': [{'va...",How many total years was Art Carney married to...,28 years,2,TEMPORAL,Is temporal
2,"[{'id': 19, 'completed_by': 1, 'result': [], '...",Which spouse was Art Carney married to the least?,Barbara Isaac,2,UNKNOWN,Not temporal
3,"[{'id': 20, 'completed_by': 1, 'result': [{'va...",How many years before he died was Art Carney m...,23,2,COUNT,Is temporal
4,"[{'id': 21, 'completed_by': 1, 'result': [{'va...",How old was Art Carney when he first got divor...,47,2,AGE,Is temporal


In [8]:
annotations["annotations"].iloc[0][0]["result"][0]["value"]["choices"][0]

'true'

In [9]:
# Extract content from very deeply nested JSON
annotations = annotations.assign(annotations=lambda x: x["annotations"].apply(lambda y: y[0]["result"]))

In [10]:
annotations = annotations.assign(
    temporal_class_correct=lambda x: x["annotations"].apply(
        lambda y: y[0]["value"]["choices"][0] if len(y) else y
    )
).drop(columns="annotations")
annotations.head()

,question,answer,table_id,answer_type,is_temporal,temporal_class_correct
0,How many years did Art Carney as actor since 1...,54 Years,2,TEMPORAL,Is temporal,true
1,How many total years was Art Carney married to...,28 years,2,TEMPORAL,Is temporal,true
2,Which spouse was Art Carney married to the least?,Barbara Isaac,2,UNKNOWN,Not temporal,[]
3,How many years before he died was Art Carney m...,23,2,COUNT,Is temporal,true
4,How old was Art Carney when he first got divor...,47,2,AGE,Is temporal,true


In [11]:
annotations.loc[annotations.loc[:, "temporal_class_correct"].str.len()==0, "temporal_class_correct"]

2       []
1058    []
Name: temporal_class_correct, dtype: object

In [12]:
annotations.loc[annotations.loc[:, "temporal_class_correct"].str.len()==0, "temporal_class_correct"] = "true"

In [13]:
# All pre-labeled questions that are wrong are "Is temporal". 
# Reason being that I already had corrected the "Not temporal" in Excel manually.
annotations.loc[annotations.loc[:, "temporal_class_correct"] == "false", "is_temporal"].unique()

array(['Is temporal'], dtype=object)

In [14]:
annotations.loc[annotations.loc[:, "temporal_class_correct"] == "false", "is_temporal"] = "Not temporal"
annotations = annotations.drop(columns="temporal_class_correct").query("is_temporal == 'Is temporal'")

In [37]:
# Identify question and answers where the expected answer is pretty clear
annotations.query(
    "~answer.str.lower().str.contains('year') "
    "and ~answer.str.lower().str.contains('month') "
    "and ~answer.str.lower().str.contains('days') "
    "and ~question.str.lower().str.startswith('how many years') "
    "and ~question.str.lower().str.startswith('for how many years') "
    "and ~question.str.lower().str.startswith('how old was') "
    "and ~question.str.lower().str.startswith('at what age') "
    "and ~question.str.lower().str.startswith('what year did') "
    "and ~question.str.lower().str.startswith('what year was') "
    "and ~question.str.lower().str.startswith('how many months') "
    "and ~question.str.lower().str.startswith('in what year') "
    "and ~question.str.lower().str.startswith('how many days') "
    "and ~question.str.lower().str.startswith('what was age') "
    "and ~question.str.lower().str.startswith('which year is') "
).loc[:, "question"].str.lower().str.split().apply(lambda x: tuple(x[:3])).value_counts()

question
(when, was, the)        61
(what, was, the)        43
(what, is, the)         11
(when, did, the)         7
(how, long, was)         4
                        ..
(around, what, year)     1
(if, mars, express)      1
(how, much, time)        1
(what, was, art)         1
(when, did, sri)         1
Name: count, Length: 97, dtype: int64

In [34]:
annotations.query(
    "question.str.lower().str.startswith('which year is') "

).loc[:, "question"]#.str.lower().str.split().apply(lambda x: tuple(x[:3])).value_counts()

581     Which year is founded of Ambassadors Group, Inc?
612                           Which year is last launch?
613                          Which year is first launch?
1042         Which year is opened of Delaware Extension?
1044         Which year is closed of Delaware Extension?
Name: question, dtype: object